<a href="https://colab.research.google.com/github/draaronv/statistical_learning/blob/master/classication_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('googledrive')
install.packages('httpuv') 
library('googledrive') 
library('httpuv')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘promises’, ‘later’




In [ ]:
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.11.0 (2021-09-26 08:30:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, inherits, isOpen, nullfile, parse,
    warnings




In [ ]:
x <- drive_get("~/statistical_learning/finalData.csv")
drive_download(x)





✔ The input `path` resolved to exactly 1 file.

File downloaded:

• finalData.csv <id: 1W1wazYCqs5Iiz6cAwFUjo9f3a3kaBxWU>

Saved locally as:

• finalData.csv



In [ ]:
rm(list = ls())

In [ ]:
install.packages("rsample")
install.packages("rpart")
install.packages("dplyr")
install.packages("rpart.plot")
install.packages("ipred")
install.packages("caret")
install.packages("caTools")
install.packages("broom")
install.packages("parallel")
install.packages("doParallel")
install.packages("FNN")
install.packages("Metrics")
install.packages("e1071")
library(e1071)
library(parallel)
library(FNN)
library(doParallel)
library(broom)
library(Metrics)
library(rsample)     # data splitting 
library(dplyr)       # data wrangling
library(rpart)       # performing regression trees
library(rpart.plot)  # plotting regression trees
library(ipred)       # bagging
library(caret)       # bagging
library(caTools)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘parallel’ is a base package, and should not be updated”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Inst

In [ ]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

# Data Initilization

In [ ]:
data1 = read.csv("finalData.csv")

In [ ]:
data2=head(data1,10000)

In [ ]:
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                  0                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 10000

[1] 10000

[1] 10000

[1] 10000

'data.frame':	7857 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 8 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 6 levels "Clear","Clouds",..: 1 1 1 1 1 1 1

## KNN Classification


In [ ]:
set.seed(1)
glm.fit1 <- train(KILLED_INJURED~., data = training,method = "knn",tuneLength = 5)
glm.fit1

k-Nearest Neighbors 

7857 samples
  13 predictor
   8 classes: '0', '1', '2', '3', '4', '5', '6', '8' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 7857, 7857, 7857, 7857, 7857, 7857, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa     
   5  0.6923705  0.15488351
   7  0.7020292  0.13529840
   9  0.7126614  0.12110338
  11  0.7191464  0.10612997
  13  0.7243807  0.08815363

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 13.

In [ ]:
pred=predict(glm.fit1,test)
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

Confusion Matrix and Statistics

    predic
true    0    1    2    3    4    5    6    8
   0 1574   19    0    0    0    0    0    0
   1  360    8    0    0    0    0    0    0
   2  110    3    0    0    0    0    0    0
   3   28    0    0    3    0    0    0    0
   4   26    0    0    1    0    0    0    0
   5    8    0    0    0    0    0    0    0
   6    2    0    0    0    0    0    0    0
   8    1    0    0    0    0    0    0    0

Overall Statistics
                                          
               Accuracy : 0.7396          
                 95% CI : (0.7205, 0.7581)
    No Information Rate : 0.9841          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.0212          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: 0 Class: 1 Class: 2 Class: 3 Class: 4 Class: 5
Sensitivity           0.74633 0.266667       N

#  Different Size

In [ ]:
## Data pre-processing
data2=head(data1,50000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
data2=data2[!is.na(data2$visibility),]
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
set.seed(1)
glm.fit1 <- train(KILLED_INJURED~., data = training, method = "knn")
glm.fit1
pred=predict(glm.fit1,test)
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 50000

[1] 49917

[1] 49917

[1] 49917

'data.frame':	39219 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 8 levels "Clear","Clouds",..: 1 1 1 1 1 1

k-Nearest Neighbors 

39219 samples
   13 predictor
   12 classes: '0', '1', '2', '3', '4', '5', '6', '7', '8', '10', '12', '18' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 39219, 39219, 39219, 39219, 39219, 39219, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa    
  5  0.7047636  0.1514548
  7  0.7155934  0.1312151
  9  0.7247987  0.1167162

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 9.

Confusion Matrix and Statistics

    predic
true    0    1    2    3    4    5    6    7    8   10   12   18
  0  7790  298   27   18    7    2    5    0    0    1    1    1
  1  1484  216   12    7    1    2    1    0    0    0    0    0
  2   448   28   19    4    0    0    0    0    0    0    0    0
  3   142   16    1    6    0    0    0    0    0    0    0    0
  4    72    3    0    0    9    0    0    0    0    0    0    0
  5    34    3    0    0    0    6    0    0    0    0    0    0
  6     9    4    0    0    0    0    0    0    0    0    0    0
  7     5    1    0    0    0    0    0    0    0    0    0    0
  8     4    0    1    0    0    0    0    0    0    0    0    0
  10    3    0    0    0    0    0    0    0    0    0    0    0
  12    0    0    0    0    0    0    0    0    0    0    3    0
  18    1    0    0    0    0    0    0    0    0    0    0    3

Overall Statistics
                                          
               Accuracy : 0.7527          
     

In [ ]:
## Data pre-processing
data2=head(data1,50000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
set.seed(1)
glm.fit1 <- train(KILLED_INJURED~., data = training, method = "knn",tuneLength=2)
glm.fit1
pred=predict(glm.fit1,test)
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 50000

[1] 49917

[1] 49917

[1] 49917

[1] 20057

'data.frame':	15758 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  82 33 56 31 35 27 38 35 39 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 1 1 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 1 4 4 4 4 4 4 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 8 levels "Clear","Clouds",..: 1 1 1 1 1 1

k-Nearest Neighbors 

15758 samples
   13 predictor
   12 classes: '0', '1', '2', '3', '4', '5', '6', '7', '8', '10', '12', '18' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 15758, 15758, 15758, 15758, 15758, 15758, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa    
  5  0.4652561  0.1816547
  7  0.4616543  0.1686608

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 5.

Confusion Matrix and Statistics

    predic
true   0   1   2   3   4   5   6   7   8  10  12  18
  0  837 671 141  44  24   7   5   0   5   0   0   0
  1  539 978 124  41  12   4   1   5   0   2   0   1
  2  176 186 144  13   2   0   0   0   0   1   0   0
  3   66  54  18  32   2   1   0   0   0   0   0   0
  4   26  31   3   4  19   0   0   0   0   0   0   0
  5    5  13   1   4   0  13   0   0   0   0   0   0
  6    7   5   2   0   0   0   7   0   0   0   0   0
  7    1   2   0   0   0   0   0   3   0   0   0   0
  8    1   0   0   0   0   0   0   0   4   0   0   0
  10   0   3   0   0   0   0   0   0   0   0   0   0
  12   0   0   0   0   0   0   0   0   0   0   3   0
  18   2   0   1   0   0   0   0   0   0   0   0   3

Overall Statistics
                                          
               Accuracy : 0.4752          
                 95% CI : (0.4602, 0.4903)
    No Information Rate : 0.452           
    P-Value [Acc > NIR] : 0.001161        
                                

In [ ]:
## Data pre-processing
data2=head(data1,50000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
set.seed(1)
glm.fit1 <- train(KILLED_INJURED~., data = training, method = "knn",tuneLength=2)
glm.fit1
pred=predict(glm.fit1,test)
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 50000

[1] 49917

[1] 49917

[1] 49917

[1] 20057

'data.frame':	15758 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  65 39 35 38 35 35 35 25 38 65 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 1 2 2 2 2 1 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 4 4 4 4 2 2 2 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 8 levels "Clear","Clouds",..: 1 1 1 1 1 1

k-Nearest Neighbors 

15758 samples
   13 predictor
   12 classes: '0', '1', '2', '3', '4', '5', '6', '7', '8', '10', '12', '18' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 15758, 15758, 15758, 15758, 15758, 15758, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa   
  5  0.4641787  0.180172
  7  0.4600544  0.166320

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 5.

Confusion Matrix and Statistics

    predic
true   0   1   2   3   4   5   6   7   8  10  12  18
  0  853 661 145  46  19   4   7   1   1   1   0   0
  1  546 985 126  37  18   2   9   1   0   2   0   0
  2  155 204 142  12   3   2   1   0   2   0   0   0
  3   50  50  11  46   4   2   0   0   0   0   0   0
  4   29  18   2   1  22   0   0   0   0   1   0   0
  5   11  14   2   2   0  15   0   0   0   0   0   0
  6    3   6   2   0   0   0   2   0   0   0   0   0
  7    3   1   0   0   0   0   0   1   0   0   0   0
  8    2   0   1   0   0   0   0   0   4   0   0   0
  10   0   1   0   0   0   0   0   0   0   2   0   0
  12   0   0   0   0   0   0   0   0   0   0   3   0
  18   0   0   0   0   0   0   0   0   0   0   0   3

Overall Statistics
                                          
               Accuracy : 0.4834          
                 95% CI : (0.4683, 0.4984)
    No Information Rate : 0.4513          
    P-Value [Acc > NIR] : 1.292e-05       
                                